In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re

In [3]:
# Укажите путь к файлу в Google Drive
file_path = '/content/drive/MyDrive/train-00000-of-00001.parquet'

# Чтение файла
df = pd.read_parquet(file_path)

# Вывод первых строк
df.head()

,image,Uniq Id,Product Name,Category,Selling Price,Model Number,About Product,Product Specification,Technical Details,Shipping Weight,Variants,Product Url,Is Amazon Seller,description
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,002e4642d3ead5ecdc9958ce0b3a5a79,"Kurio Glow Smartwatch for Kids with Bluetooth,...",Toys & Games | Kids' Electronics | Electronic ...,$31.30,C17515,Make sure this fits by entering your model num...,ProductDimensions:5x3x12inches|ItemWeight:7.2o...,Color:Blue show up to 2 reviews by default Thi...,7.2 ounces,https://www.amazon.com/Kurio-Smartwatch-Blueto...,https://www.amazon.com/Kurio-Smartwatch-Blueto...,Y,"Kurio Glow Smartwatch: Fun, Safe & Educational..."
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,009359198555dde1543d94568183703c,Star Ace Toys Harry Potter & The Prisoner of A...,None,$174.99,SA8011B,Make sure this fits by entering your model num...,ProductDimensions:2.5x1x9inches|ItemWeight:1.4...,From Star Ace Toys. Many fans would say that H...,1.43 pounds,None,https://www.amazon.com/Star-Ace-Toys-Prisoner-...,Y,Relive the magic! Star Ace Toys' 1/8 scale Ha...
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,00cb3b80482712567c2180767ec28a6a,Barbie Fashionistas Doll Wear Your Heart,Toys & Games | Dolls & Accessories | Dolls,$15.99,FJF44,Make sure this fits by entering your model num...,ProductDimensions:2.1x4.5x12.8inches|ItemWeigh...,Go to your orders and start the return Select ...,4.2 ounces,None,https://www.amazon.com/Barbie-FJF44-Love-Fashi...,Y,Express your style with Barbie Fashionistas Do...
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,00cce525ebf9181ebfba30dc5ca936fd,Redcat Racing Aluminum Rear Lower Suspension A...,Toys & Games | Hobbies | Remote & App Controll...,$14.40,06049B,"Aluminum Rear Lower Suspension Arms, Blue (2pc...",ProductDimensions:1.5x3.5x0.2inches|ItemWeight...,2.4 ounces (View shipping rates and policies) ...,2.4 ounces,None,https://www.amazon.com/Redcat-Racing-Aluminum-...,Y,Upgrade your Redcat Racing vehicle's performan...
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,015cc42a8e93b15bcea9425d63ecbbd9,"Tru-Ray Heavyweight Construction Paper Pad, 10...","Arts, Crafts & Sewing | Crafting | Paper & Pap...",$10.10,6592,Make sure this fits by entering your model num...,ASIN:B01ELJGWKW|ShippingWeight:1pounds(Viewshi...,Go to your orders and start the return Select ...,1 pounds,https://www.amazon.com/Tru-Ray-Heavyweight-Con...,https://www.amazon.com/Tru-Ray-Heavyweight-Con...,Y,Unleash your creativity with Tru-Ray Heavyweig...


In [4]:
# Список ненужных столбцов (замени на актуальные названия в твоём случае)
columns_to_drop = [
    'image', 'Uniq Id',
    'Model Number', 'Shipping Weight', 'Variants', 'Is Amazon Seller', 'Product Specification'
]

# Удалим столбцы, если они есть
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

In [5]:
# Отключаем перенос текста в ячейках (чтобы не было многоточий ...)
pd.set_option('display.width', None)  # Автоматическая ширина
pd.set_option('display.max_colwidth', None)  # Показывать полный текст в ячейках

In [6]:
df

,Product Name,Category,Selling Price,About Product,Technical Details,Product Url,description
0,"Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue",Toys & Games | Kids' Electronics | Electronic Learning Toys,$31.30,"Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Color:Blue show up to 2 reviews by default This sleek, hi-tech Bluetooth Smartwatch is made specifically for kids, and it's packed with apps and games for out-of-the-box fun! Take selfies and videos, play single and two-player games, message friends, listen to music, plus everything you need in a smartwatch– activity tracker, alarm/stopwatch, calculator, calendar and so much more! Plus, parents can add vital information like blood type and allergies to an 'in case of an emergency' (I. C. E. ) app | 7.2 ounces (View shipping rates and policies)",https://www.amazon.com/Kurio-Smartwatch-Bluetooth-Camera-Games/dp/B07TFD5D55,"Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow"
1,"Star Ace Toys Harry Potter & The Prisoner of Azkaban: Harry Potter with Dobby 1: 8 Scale Collectible Action Figure, Multicolor SA8011B",None,$174.99,"Make sure this fits by entering your model number. | From the classic film | Presents Harry as he appeared in the Prisoner Of Azkaban | Comes with the monster book of monsters, hedwig the owl, the Marauder's map and his wand | Includes a figure of the house Elf Dobby | Figure is in 1: 8 scale","From Star Ace Toys. Many fans would say that Harry Potter and the Prisoner Of Azkaban is their favorite film of the series. Directed by alfonso cuarón, this film is darker and more adult than the previous two films. Star ace is proud to present Harry Potter as he appeared in this movie in his Hogwarts school robes. He also comes with the monster book of monsters, Hedwig the owl, the Marauder's map and his wand. This special set also includes the house elf Dobby! | 1.43 pounds (View shipping rates and policies)",https://www.amazon.com/Star-Ace-Toys-Prisoner-Azkaban/dp/B07KMXGSXF,"Relive the magic! Star Ace Toys' 1/8 scale Harry Potter & Dobby collectible figure (SA8011B) from *Prisoner of Azkaban* is here. Highly detailed, this action figure features Harry and his loyal house-elf, Dobby. A must-have for Harry Potter collectors!"
2,Barbie Fashionistas Doll Wear Your Heart,Toys & Games | Dolls & Accessories | Dolls,$15.99,"Make sure this fits by entering your model number. | Barbie Fashionistas doll loves this outfit -- the pink Sweatshirt dress has a cool ""love"" typographic with a sheen touch | Boots and a choker complete the look | Her long hair is right on trend in high pigtails | More variety makes collecting Barbie Fashionistas dolls even more fun | Collect them all (each sold separately, subject to availability)","Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | show up to 2 reviews by default Every Barbie fashionistas doll has her own look from cas

In [7]:
# Очистка Selling Price: удалим символы валют и запятые, приведём к float
df['Selling Price'] = (
    df['Selling Price']
    .astype(str)
    .replace(r'[₹$,]', '', regex=True)
    .replace(r',', '', regex=True)
    .astype(str)
    .str.strip()
)
df['Selling Price'] = pd.to_numeric(df['Selling Price'], errors='coerce')

In [8]:
df

,Product Name,Category,Selling Price,About Product,Technical Details,Product Url,description
0,"Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue",Toys & Games | Kids' Electronics | Electronic Learning Toys,31.30,"Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Color:Blue show up to 2 reviews by default This sleek, hi-tech Bluetooth Smartwatch is made specifically for kids, and it's packed with apps and games for out-of-the-box fun! Take selfies and videos, play single and two-player games, message friends, listen to music, plus everything you need in a smartwatch– activity tracker, alarm/stopwatch, calculator, calendar and so much more! Plus, parents can add vital information like blood type and allergies to an 'in case of an emergency' (I. C. E. ) app | 7.2 ounces (View shipping rates and policies)",https://www.amazon.com/Kurio-Smartwatch-Bluetooth-Camera-Games/dp/B07TFD5D55,"Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow"
1,"Star Ace Toys Harry Potter & The Prisoner of Azkaban: Harry Potter with Dobby 1: 8 Scale Collectible Action Figure, Multicolor SA8011B",None,174.99,"Make sure this fits by entering your model number. | From the classic film | Presents Harry as he appeared in the Prisoner Of Azkaban | Comes with the monster book of monsters, hedwig the owl, the Marauder's map and his wand | Includes a figure of the house Elf Dobby | Figure is in 1: 8 scale","From Star Ace Toys. Many fans would say that Harry Potter and the Prisoner Of Azkaban is their favorite film of the series. Directed by alfonso cuarón, this film is darker and more adult than the previous two films. Star ace is proud to present Harry Potter as he appeared in this movie in his Hogwarts school robes. He also comes with the monster book of monsters, Hedwig the owl, the Marauder's map and his wand. This special set also includes the house elf Dobby! | 1.43 pounds (View shipping rates and policies)",https://www.amazon.com/Star-Ace-Toys-Prisoner-Azkaban/dp/B07KMXGSXF,"Relive the magic! Star Ace Toys' 1/8 scale Harry Potter & Dobby collectible figure (SA8011B) from *Prisoner of Azkaban* is here. Highly detailed, this action figure features Harry and his loyal house-elf, Dobby. A must-have for Harry Potter collectors!"
2,Barbie Fashionistas Doll Wear Your Heart,Toys & Games | Dolls & Accessories | Dolls,15.99,"Make sure this fits by entering your model number. | Barbie Fashionistas doll loves this outfit -- the pink Sweatshirt dress has a cool ""love"" typographic with a sheen touch | Boots and a choker complete the look | Her long hair is right on trend in high pigtails | More variety makes collecting Barbie Fashionistas dolls even more fun | Collect them all (each sold separately, subject to availability)","Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | show up to 2 reviews by default Every Barbie fashionistas doll has her own look from casual

In [9]:
nan_count = df.isna().sum()
print(f"Количество NaN в столбце: {nan_count}")

Количество NaN в столбце: Product Name           0
Category             120
Selling Price         63
About Product         35
Technical Details     92
Product Url            0
description            0
dtype: int64


In [10]:
# Сброс индекса (если нужно)
df = df.reset_index(drop=True)

# Векторизованное заполнение NaN
df['Selling Price'] = df['Selling Price'].fillna(
    (df['Selling Price'].ffill() + df['Selling Price'].bfill()) / 2
)
df['Selling Price'] = df['Selling Price'].fillna(df['Selling Price'].ffill())
df['Selling Price'] = df['Selling Price'].fillna(df['Selling Price'].bfill())

In [11]:
df = df.drop_duplicates()

In [12]:
df = df.dropna()

In [13]:
df

,Product Name,Category,Selling Price,About Product,Technical Details,Product Url,description
0,"Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue",Toys & Games | Kids' Electronics | Electronic Learning Toys,31.30,"Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Color:Blue show up to 2 reviews by default This sleek, hi-tech Bluetooth Smartwatch is made specifically for kids, and it's packed with apps and games for out-of-the-box fun! Take selfies and videos, play single and two-player games, message friends, listen to music, plus everything you need in a smartwatch– activity tracker, alarm/stopwatch, calculator, calendar and so much more! Plus, parents can add vital information like blood type and allergies to an 'in case of an emergency' (I. C. E. ) app | 7.2 ounces (View shipping rates and policies)",https://www.amazon.com/Kurio-Smartwatch-Bluetooth-Camera-Games/dp/B07TFD5D55,"Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow"
2,Barbie Fashionistas Doll Wear Your Heart,Toys & Games | Dolls & Accessories | Dolls,15.99,"Make sure this fits by entering your model number. | Barbie Fashionistas doll loves this outfit -- the pink Sweatshirt dress has a cool ""love"" typographic with a sheen touch | Boots and a choker complete the look | Her long hair is right on trend in high pigtails | More variety makes collecting Barbie Fashionistas dolls even more fun | Collect them all (each sold separately, subject to availability)","Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | show up to 2 reviews by default Every Barbie fashionistas doll has her own look from casually cool to boho bold, all fashions are inspired by the latest trends. Collect them all to explore countless styles, fashions, shoes and accessories. The latest line of Barbie fashionistas dolls includes four body types, nine skin tones, 13 eye colors, 13 hairstyles and countless on-trend fashions and accessories. With these additions, girls everywhere will have infinitely more ways to play out their stories and spark their imaginations through Barbie -- because with Barbie, you can be anything! each sold separately, subject to availability. Dolls cannot stand alone. Clothing is designed to mix and match with dolls of the same body Type; select pieces can be shared across the line. Flat shoes fit dolls with articulated ankles or flat feet. Colors and decorations may vary. | 4.2 ounces (View shipping rates and policies)",https://www.amazon.com/Barbie-FJF44-Love-Fashion-Doll/dp/B0751ZH2ZT,"Express your style with Barbie Fashionistas Doll Wear Your Heart! This fashionable doll boasts a unique outfit and accessories, perfect for imaginative play. A great gift for kids aged 3+. Collect them all! #Barbie #Fashionistas #Doll #Toys #GirlsToys #FashionDoll #Play"
3,"Redcat Racing Aluminum Rear Lower Suspension Arms (2 Piece), Blue",Toys & Games | Hobbies | Remote & App Controll

In [14]:
#Импортируем TfIdfVectorizer из scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Определим векторизатор TF-IDF. Удалим все английские стоп-слова, такие как 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Все NaN заменим пустыми строками
df['description'] = df['description'].fillna('')

#Построми матрицу TF-IDF
tfidf_matrix = tfidf.fit_transform(df['description'])

#Выведем размерность этой матрицы
tfidf_matrix.shape

(1118, 5654)

In [15]:
from sklearn.metrics.pairwise import linear_kernel

# Вычисляем косинусное расстояние
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [16]:
df = df.rename(columns={"Product Name": "name"})

In [17]:
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

In [18]:
def get_recommendations(name, cosine_sim=cosine_sim):
    idx = indices[name]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    product_indices = [i[0] for i in sim_scores]

    return df['name'].iloc[product_indices]

In [19]:
get_recommendations('Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue')

,name
216,"Rubies Monster High Fright Camera Action Draculaura Costume, Child Medium"
469,Rhode Island Novelty ~ 12 ~ Foam Dice ~ 1.5 Inch ~ Games Math Teaching ~ New
741,Junior Learning Touchtronic Placeholder Toy
1300,Swing-N-Slide Blue Child Seat
1092,Learning Resources Canadian Currency-X-Change
985,"Educational Insights Design & Drill See-Through Creative Workshop - Drill Toy, STEM & Construction, Building Toy"
695,"Kev & Cooper Playtime Collection Road Map Educational Area Rug - 3'3"" x 4'7"""
896,"Deny Designs Iveta Abolina Winter Blue Throw Pillow, Blue, Extra Large/26 x 26"
175,eKids Paw Patrol Bluetooth Portable MP3 Karaoke Machine Player with LED Display Store Hours of Music with Built in Memory Sing Along Using The Real Working Microphone USB Port to Expand Your Content
568,"Starplay Childrens Slide, Red/Blue"


In [20]:
# Объединение столбцов description и About Product
df['combined_text'] = df['description'] + " " + df['About Product']

In [21]:
# Предобработка текста
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
def preprocess_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Удаление пунктуации
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Лемматизация и удаление стоп-слов
    return ' '.join(words)

df['processed_text'] = df['combined_text'].apply(preprocess_text)

In [23]:
#Импортируем TfIdfVectorizer из scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Определим векторизатор TF-IDF. Удалим все английские стоп-слова, такие как 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Все NaN заменим пустыми строками
df['processed_text'] = df['processed_text'].fillna('')

#Построми матрицу TF-IDF
tfidf_matrix = tfidf.fit_transform(df['processed_text'])

#Выведем размерность этой матрицы
tfidf_matrix.shape

(1118, 8667)

In [24]:
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

In [25]:
def get_recommendations(name, cosine_sim=cosine_sim):
    idx = indices[name]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    product_indices = [i[0] for i in sim_scores]

    return df['name'].iloc[product_indices]

In [26]:
get_recommendations('Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue')

,name
216,"Rubies Monster High Fright Camera Action Draculaura Costume, Child Medium"
469,Rhode Island Novelty ~ 12 ~ Foam Dice ~ 1.5 Inch ~ Games Math Teaching ~ New
741,Junior Learning Touchtronic Placeholder Toy
1300,Swing-N-Slide Blue Child Seat
1092,Learning Resources Canadian Currency-X-Change
985,"Educational Insights Design & Drill See-Through Creative Workshop - Drill Toy, STEM & Construction, Building Toy"
695,"Kev & Cooper Playtime Collection Road Map Educational Area Rug - 3'3"" x 4'7"""
896,"Deny Designs Iveta Abolina Winter Blue Throw Pillow, Blue, Extra Large/26 x 26"
175,eKids Paw Patrol Bluetooth Portable MP3 Karaoke Machine Player with LED Display Store Hours of Music with Built in Memory Sing Along Using The Real Working Microphone USB Port to Expand Your Content
568,"Starplay Childrens Slide, Red/Blue"


In [27]:
get_recommendations('Junior Learning Touchtronic Placeholder Toy')

,name
183,Stamps by Impression ST 0766a Lion Rubber Stamp
456,Prima Marketing Metal Dies for JN Stamp Audrey
338,"Graphic 45 4501829 Love Notes Stickers Craft Paper, Multi"
615,"Franklin Sports 8.5"" Rubber Tetherball"
96,"Eureka Back to School Red and White Dr. Seuss Striped Paper Letters, 130pc, 1.31'' x 1.75''"
731,Name Stampers Arianna Rubber Stamps
85,Essentials Sticker
648,American Crafts 8 Roll Washi Tape 3 Creative Devotion
956,"Barker Creek Chevron Black Tie Affair, 30-Count, Self-Adhesive (LL-1233)"
161,"Creativity Street Dinosaurs Embossed Paper, 24-Pieces"


In [28]:
df


,name,Category,Selling Price,About Product,Technical Details,Product Url,description,combined_text,processed_text
0,"Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue",Toys & Games | Kids' Electronics | Electronic Learning Toys,31.30,"Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Color:Blue show up to 2 reviews by default This sleek, hi-tech Bluetooth Smartwatch is made specifically for kids, and it's packed with apps and games for out-of-the-box fun! Take selfies and videos, play single and two-player games, message friends, listen to music, plus everything you need in a smartwatch– activity tracker, alarm/stopwatch, calculator, calendar and so much more! Plus, parents can add vital information like blood type and allergies to an 'in case of an emergency' (I. C. E. ) app | 7.2 ounces (View shipping rates and policies)",https://www.amazon.com/Kurio-Smartwatch-Bluetooth-Camera-Games/dp/B07TFD5D55,"Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow","Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!",kurio glow smartwatch fun safe educational kid smartwatch boast bluetooth connectivity builtin apps game camera vibrant blue design perfect learning play kidssmartwatch kidselectronics educationaltoys kurioglow make sure fit entering model number kurio watch glow real bluetooth smartwatch built especially kid packed apps game get glow new lightup feature turn game activity colorful fun kurio watch glow includes brandnew game light effect including little dragon virtual pet colorchanging mood sensor play single twoplayer game one watch connect two watch together via bluetooth plus motionsensitive game get kid moving take fun selfies frontfacing camera decorate filter frame sticker plus everything need smartwatch activity tracker music player alarmstopwatch calculator calendar much scratch resistant splashproof suitable kid age
2,Barbie Fashionistas Doll Wear Your Heart,Toys & Games | D

In [36]:
df

,id,name,Category,Selling Price,About Product,Technical Details,Product Url,description,combined_text,processed_text
0,None,"Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue",Toys & Games | Kids' Electronics | Electronic Learning Toys,31.30,"Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Color:Blue show up to 2 reviews by default This sleek, hi-tech Bluetooth Smartwatch is made specifically for kids, and it's packed with apps and games for out-of-the-box fun! Take selfies and videos, play single and two-player games, message friends, listen to music, plus everything you need in a smartwatch– activity tracker, alarm/stopwatch, calculator, calendar and so much more! Plus, parents can add vital information like blood type and allergies to an 'in case of an emergency' (I. C. E. ) app | 7.2 ounces (View shipping rates and policies)",https://www.amazon.com/Kurio-Smartwatch-Bluetooth-Camera-Games/dp/B07TFD5D55,"Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow","Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!",kurio glow smartwatch fun safe educational kid smartwatch boast bluetooth connectivity builtin apps game camera vibrant blue design perfect learning play kidssmartwatch kidselectronics educationaltoys kurioglow make sure fit entering model number kurio watch glow real bluetooth smartwatch built especially kid packed apps game get glow new lightup feature turn game activity colorful fun kurio watch glow includes brandnew game light effect including little dragon virtual pet colorchanging mood sensor play single twoplayer game one watch connect two watch together via bluetooth plus motionsensitive game get kid moving take fun selfies frontfacing camera decorate filter frame sticker plus everything need smartwatch activity tracker music player alarmstopwatch calculator calendar much scratch resistant splashproof suitable kid age
2,None,Barbie Fashionistas Doll Wear Your Heart,Toy

In [37]:
# from google.colab import files

# # Сохраняем DataFrame во временный файл и скачиваем
# df.to_csv('bd_vkr.csv', index=False)
# files.download('bd_vkr.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>